In [1]:
# Imports
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv

# Environment setup
load_dotenv()
deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']
language="python"

#Initialize AzureOpenAI client
client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_KEY'],  
  api_version = "2023-12-01-preview"
  )

In [3]:
def select_machine_family(machineKind):
    if machineKind == "cutting":
        return "cut"
    elif machineKind == "packing":
        return "pack"
    elif machineKind == "soldering":
        return "solder"
    else:
        return ""

def select_machine_family(machine_family:str, cutMode:str) -> str:    
    if machine_family == "cut":
        if cutMode=="vertical":
            return "cut_vertical"
        elif cutMode=="horizontal":
            return "cut_horizontal"
        else:
            return "cut_round"
    #TODO: add other machine families
    else:
        return ""
    
def select_external_library_for_cutting(machineModel:str, operation:str) -> str:    
    if machineModel == "SE-0001":
        if operation=="cut_vertical":
            return  '''
                The library package "machine_cut_v" contains functions related to material cutting.
                
                Here's the list of available functions:
                         
                # Cut the material vertically from the point (beginX,beginY) to the point (endX,endY)
                verticalCut(beginX, beginY, endX, endY)
                
                # Cut the material vertically from the point (beginX,beginY) to the point (endX,endY) in a zigzag pattern
                verticalCutZZ(beginX, beginY, endX, endY)        
                '''
        elif operation=="cut_horizontal":
            return  '''
                The library package "machine_cut_h" contains functions related to material cutting.
                
                Here's the list of available functions:
                         
                # Cut the material horizontally from the point (beginX,beginY) to the point (endX,endY)
                HCut(beginX, beginY, endX, endY)
                
                # Cut the material horizontally from the point (beginX,beginY) to the point (endX,endY) in a zigzag pattern
                HZZCut(beginX, beginY, endX, endY)
                '''
        elif operation=="cut_round":
            return  '''
                The library package "machine_cut_r" contains functions related to material cutting.
                
                Here's the list of available functions:
                         
                # Creates a round cut where point (beginX,beginY) is the center and radius is the radius of the circle
                RoundCut(beginX, beginY, radius)
                
                # Creates a round cut where point (beginX,beginY) is the center, radius is the radius of the circle and border follows a zigzag pattern
                RoundCutZigZag(beginX, beginY, endX, endY)
                '''
        else:
            return "cut_horizontal"
    #TODO: add other machine families
    else:
        return ""

In [4]:
functions = [
   {
      "name":"select_machine_family",
      "description":"Select the family of machine to use for library selection",
      "parameters": {
         "type":"object",
         "properties":{
            "machineKind":{
               "type":"string",               
               "description": "The family of the machine, can be one of the following: cutting, packing, soldering"
            },
         "required":["machineKind"]
         },
      },
   },  
]

This is the system message that sets the chat behavior for the whole conversation

In [5]:
# system message
system_message = f'''
You are a an expert in generating code using exclusively the {language} language.
Based on the user request you will generate the code for the user following the classic {language} conventions and commenting the code so that it will be easily readable favoring semplicity over complexity.
The generated code should be syntactically correct and should be able to run without errors and the it should include the following features:
- All the required imports should be included and listed at the top of the file.
- The code should be executed via a main function.
- Do not generate additional ouput comments other than the inline with the code.

In order to generate compliant code you might need to use external libraries, use function provided in the prompt to request detailed information about them.

'''

#### Select the type of prompt you want to use and then run the cell that invokes the model.

The following prompt instruct the generator to emit code to turn/off lights providing a reference to a fictional "machine_io" external library and listing exported functions. 

**Expected result**: Have a valid code generated.

In [7]:
# initial message
initial_message = f'''
I want to generate a program to control a machine model SE01 via a programmable logic controller (PLC) to cut a material, vertically, following a zigzag pattern from the point (10,10) to the point (100,100).

Here's the criteria to follow for picking the correct function/s:

-Use a specific external function only if:
    - The function name seems to be designed for the specific use case (e.g. blow_air() in case the code should blow air)
    - The optional function comments or related description seems to match the functionality.
    - Associated function comments or related documentation seems to describe the required functionality (e.g. turn_on_machine() is associated with a comment or documentaion that says "This function turns on the machine") despite the name of the function itself seems to disagree:
        Examples:
        - the name of the function is: turn_on_system() but the comment says "turns on the machine" in this case the function should be used when the code should turn on the machine.
        - the name of the function is: HSI(sec) and the comment says "It heats the welding head for the specified amount of seconds" in this case the function should be used when the code is supposed to heat a soldering head.
    - Favor the description of the function over the name of the function, to decide if the function should be used or not.
    - The documentation indicates that the function can be used for the required functionality:
        Examples of when it should NOT be use:
        - Documentation says "This function can be used to turn on the machine (deprecated)" in this case the function should be used because deprecated.
        - Documentation indicates other better alternatives like "This function can be used to turn on the machine (note: use turn_on_machine2() instead)" in this case the suggested function turn_on_machine2() should be picked.
        
    
-do not use the external library functions if:
    - the name of the function seems to be designed for other specific use cases (e.g. turn_on_machine() seems to not be designed to blow air)    
if it appears to have external functions that overlaps in functionality (e.g. blow_air_for(seconds) and blow_air_on() and blow_air_off()) favor the one that result in lesser generated code for the requested functionality.
if none of the external libraries offer a function for the required functionality, insert a comment similar to the following one:

    # NOTE: no external function found to <replace with name the operation to achieve (e.g. 'blow air')>
    <insert a comment with a pseudo library function call, possibly using a meaningful name (e.g. turn_on_machine in case required functionality is to turn on the machine instead of foo() or xxx()) including required parameters that can be used to achieve the required functionality>
    
    the comment shold be placed before the pseudo library function call and should be indented with the same indentation of the pseudo library function call.
    the pseudo library function call should be commented out.

'''

The following prompt instruct the generator to emit code to turn/off lights providing a reference to a fictional "machine_io" external library, listing exported functions but none of them is valid to turn/off the light that's why the prompt include instructions to emit stub code in alternative.  

**Expected result**: Have function call replaced with commented out pseudo calls.

In [8]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [
     {'role': 'system', 'content': system_message},
     {'role': 'user', 'content': initial_message}
     ],
 temperature=0.3,
 max_tokens=800,
 top_p=0.95,
 frequency_penalty=0,
 presence_penalty=0,
 stop=None,
 tools=functions,
 tool_choice="auto"
)

answer=openai_response1.choices[0].message.content 
print (answer)

BadRequestError: Error code: 400 - {'error': {'message': 'Unrecognized request arguments supplied: tool_choice, tools', 'type': 'invalid_request_error', 'param': None, 'code': None}}